In [17]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from quantuminspire.qiskit import QI
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
import utils as u

In [18]:
u.set_auth()

In [19]:
def measurement_error_mitigation(qubits, qubit_list, raw_counts):
        """
        qubits = INT
        qubit_list = array
        raw_counts = dict
        """
        # create calibration circuits
        qr = QuantumRegister(qubits)
        meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='cal')
        # create calibration matrix by running calibration
        backend = QI.get_backend('QX single-node simulator') # replace 'with Starmon-5' later
        job = qiskit.execute(meas_calibs, backend=backend, shots=1024)
        cal_results = job.result()
        meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='cal')
        print(meas_fitter.cal_matrix)
        # apply measurement error mitigation via calibration matrix ####
        meas_filter = meas_fitter.filter
        mitigated_results = meas_filter.apply(raw_counts)
        mitigated_counts = mitigated_results.get_counts(0)
        return migitated_counts